In [4]:
!pip install sentence_transformers pinecone-client datasets seaborn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.9 MB

In [1]:
!pip install --upgrade torch torchvision

In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="0";
os.environ["CUDA_LAUNCH_BLOCKING"]="1";


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,TrainingArguments
)

In [3]:


class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [4]:
def Remove_line(text):
  new=text.replace('\n', ' ')
  return new

In [5]:
# load the dataset and convert to pandas dataframe
df_tain = load_dataset("fhamborg/news_sentiment_newsmtsc",split="train").to_pandas()

In [6]:
df_validation = load_dataset( "fhamborg/news_sentiment_newsmtsc",split="validation").to_pandas()

In [7]:
df_test = load_dataset("fhamborg/news_sentiment_newsmtsc",split="test").to_pandas()

In [8]:
df_tain["sentence"]=df_tain["sentence"].apply(Remove_line)
df_validation["sentence"]=df_validation["sentence"].apply(Remove_line)
df_test["sentence"]=df_test["sentence"].apply(Remove_line)


In [9]:
df_tain["sentence"][14]

'It was a familiar morning for Mr. Trump on his favored medium — and for a nation that had, for a little more than a week, gone without the president’s stream-of-consciousness missives.'

In [10]:
df_tain.polarity.value_counts()

-1    3316
 0    3028
 1    2395
Name: polarity, dtype: int64

In [11]:
df_test["sentence"][0]

'Though we do not know what other items Seth may have had in his possession, his watch, phone, wallet and necklace were not stolen.'

In [12]:
df_test

,mention,polarity,from,to,sentence,id
0,Seth,0,39,43,Though we do not know what other items Seth ma...,polusa_v1_4307505_-1_11_Seth_39_43
1,He,1,0,2,He won the Academy Award for Best Supporting A...,allsides_4276_1784_7_Beloved actor Robin Willi...
2,""" Powell",1,44,52,"""Yup, the whole birther movement was racist,"" ...",allsides_1810_737_6_Powell_44_52
3,Ross,0,0,4,Ross's appointment will require a Senate confi...,allsides_1593_643_5_Wilbur Ross_0_4
4,Andrews,0,27,34,Trustees Simone Boutet and Andrews voted no ea...,polusa_v1_59535720_-1_32_Trustee Deno Andrews_...
...,...,...,...,...,...,...
798,Poe,0,91,94,To have her now say she doesn't support our ri...,allsides_4841_2036_10_Poe_91_94
799,Obama,1,98,103,But some observers suggested it was tone-deaf ...,allsides_2495_1015_21_Obama_98_103
800,Cox,0,95,98,“It is my opinion that you are a key conspirat...,polusa_v1_39150634_-1_5_Sean Cox_95_98
801,Schmidt,-1,104,111,“It is my opinion that you are a key conspirat...,polusa_v1_39150634_-1_5_Schmidt_104_111


In [13]:
# clean_1_test = df_test.groupby('polarity', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
# clean_1_test = clean_1_test.sample(frac=1, random_state=42)
# clean_1_test.dropna(subset=['sentence'], inplace=True)
# clean_1_test

# Read Gold data

In [14]:
gold_data=pd.read_csv("https://raw.githubusercontent.com/AnasElbattra/News-Understanding-NLP/main/Sentiment%20Analysis/Data/gold-dataset-sinha-khandait.csv")
gold_data

,Dates,URL,News,Price Direction Up,Price Direction Constant,Price Direction Down,Asset Comparision,Past Information,Future Information,Price Sentiment
0,28-01-2016,http://www.marketwatch.com/story/april-gold-do...,"april gold down 20 cents to settle at $1,116.1...",0,0,1,0,1,0,negative
1,13-09-2017,http://www.marketwatch.com/story/gold-prices-s...,gold suffers third straight daily decline,0,0,1,0,1,0,negative
2,26-07-2016,http://www.marketwatch.com/story/gold-futures-...,Gold futures edge up after two-session decline,1,0,0,0,1,0,positive
3,28-02-2018,https://www.metalsdaily.com/link/277199/dent-r...,dent research : is gold's day in the sun comin...,0,0,0,0,0,1,none
4,06-09-2017,http://www.marketwatch.com/story/gold-steadies...,"Gold snaps three-day rally as Trump, lawmakers...",0,0,1,0,1,0,negative
...,...,...,...,...,...,...,...,...,...,...
10565,07-01-2013,https://www.moneycontrol.com/news/business/mar...,gold seen falling from 3-week high this week,0,0,1,0,1,0,negative
10566,27-09-2018,https://www.metalsdaily.com/link/284468/domini...,dominic frisby : now looks like a good time to...,1,0,0,0,0,1,positive
10567,03-03-2017,https://www.thehindubusinessline.com/markets/g...,Gold heading for worst week since November on ...,0,0,1,0,1,0,negative
10568,11-06-2008,http://www.marketwatch.com/story/august-gold-u...,august gold up $7.60 at $878.80 an ounce on nymex,1,0,0,0,1,0,positive


In [15]:
gold_data["Price Sentiment"].value_counts()

positive    4412
negative    3814
none        1968
neutral      376
Name: Price Sentiment, dtype: int64

In [16]:
filter_gold_data=gold_data[gold_data["Price Sentiment"]!="none"]

In [17]:
filter_gold_data["Price Sentiment"].value_counts()

positive    4412
negative    3814
neutral      376
Name: Price Sentiment, dtype: int64

In [18]:
minority_class = filter_gold_data[filter_gold_data['Price Sentiment'] == 'neutral']

In [19]:
majority_class_positive = filter_gold_data[filter_gold_data['Price Sentiment'] == 'positive']
downsampled_majority = majority_class_positive.sample(n=376, random_state=42)

majority_class_negative = filter_gold_data[filter_gold_data['Price Sentiment'] == 'negative']
downsampled_majority_negative = majority_class_negative.sample(n=376, random_state=42)

In [20]:
balanced_gold_data = pd.concat([minority_class, downsampled_majority,downsampled_majority_negative])
balanced_gold_data = balanced_gold_data.sample(frac=1, random_state=42)

In [21]:
balanced_gold_data["Price Sentiment"].value_counts()

negative    376
positive    376
neutral     376
Name: Price Sentiment, dtype: int64

In [22]:
balanced_gold_data = balanced_gold_data.rename(columns={'News': 'sentence', 'Price Sentiment': 'polarity'})


In [23]:
balanced_gold_data["polarity"]=balanced_gold_data.polarity.map({
    "neutral": 0,
    "positive": 1,
    "negative": -1
})

In [24]:
balanced_gold_data["polarity"].value_counts()

-1    376
 1    376
 0    376
Name: polarity, dtype: int64

In [25]:
# Splitting the data into training, validation, and test sets
train_gold, temp = train_test_split(balanced_gold_data, test_size=0.2, random_state=42, stratify=balanced_gold_data["polarity"])
val_gold, test_gold = train_test_split(temp, test_size=0.5, random_state=42, stratify=temp["polarity"])

In [26]:
# # Split the data into a training set (80%) and a temporary set (20%)
# X_train_gold, X_temp, y_train_gold, y_temp = train_test_split(balanced_gold_data["sentence"], balanced_gold_data["polarity"], test_size=0.2, random_state=42,stratify= balanced_gold_data["polarity"])

# # Now, further split the temporary set into a training set (80%) and a validation set (20%)
# X_val_gold, X_test_gold, y_val_gold, y_test_gold = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42,stratify= y_temp)

In [27]:
train_gold.shape

(902, 10)

In [28]:
train_gold

,Dates,URL,sentence,Price Direction Up,Price Direction Constant,Price Direction Down,Asset Comparision,Past Information,Future Information,polarity
7310,08-12-2017,https://www.metalsdaily.com/link/274711/gold-s...,"gold slides back closer to over 4-month lows, ...",0,0,1,0,1,0,-1
9423,10-07-2017,http://www.marketwatch.com/story/gold-slips-he...,Gold prices claw back from their lowest level ...,1,0,0,0,1,0,1
7601,18-04-2017,http://www.marketwatch.com/story/june-gold-hol...,"June gold holds 0.3% at $1,287.70/oz.",0,1,0,0,1,0,0
2279,01-03-2017,http://www.marketwatch.com/story/gold-futures-...,Gold futures end the session at lowest level i...,0,0,1,0,1,0,-1
7961,16-12-2014,http://www.investing.com/news/commodities-news...,Gold loses ground after market shrugs off hous...,0,0,1,0,1,0,-1
...,...,...,...,...,...,...,...,...,...,...
10117,02-09-2010,http://www.marketwatch.com/story/crude-oil-dow...,"crude oil down, gold futures up after jobless ...",1,0,0,1,1,0,1
1410,20-09-2007,http://www.marketwatch.com/story/gold-futures-...,gold futures climb past $740 in morning dealings,1,0,0,0,1,0,1
3699,06-09-2005,http://www.marketwatch.com/story/philadelphia-...,philadelphia gold/silver index falls 0.5% to c...,0,0,1,1,1,0,-1
6256,19-04-2017,http://www.marketwatch.com/story/gold-falls-in...,Gold falls in electronic trade after Fed Beige...,0,0,1,0,1,0,-1


# read  sentiment_annotated_data

In [29]:
sentiment_annotated_data=pd.read_csv("https://raw.githubusercontent.com/AnasElbattra/News-Understanding-NLP/main/Sentiment%20Analysis/Data/sentiment_annotated_with_texts.csv")
sentiment_annotated_data

,published_at,ticker,true_sentiment,title,author,url,source,text,finbert_sentiment,finbert_sent_score
0,2023-01-12 07:47:00,EURCHF,Positive,Euro to benefit from the ECBs pronounced hawki...,FXStreet Insights Team,https://www.fxstreet.com/news/euro-to-benefit-...,FX Street,The Euro was able to appreciate particularly s...,Positive,0.85
1,2023-01-12 10:34:00,EURCHF,Positive,EURCHF Trend higher may remain in place – ING,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-trend-hi...,FX Street,EUR/CHF yesterday broke above 1.00. Economists...,Positive,0.51
2,2023-01-12 11:40:00,EURCHF,Neutral,Does a jump in EURCHF point to a break above 1...,FXStreet Insights Team,https://www.fxstreet.com/news/does-a-jump-in-e...,FX Street,EUR/CHF vaults parity for the first time since...,Neutral,0.37
3,2023-01-12 15:32:00,EURCHF,Positive,EURCHF could extend its advance back to levels...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-could-ex...,FX Street,EUR/CHF climbs back above parity. Economists a...,Positive,0.64
4,2023-01-13 11:37:00,EURCHF,Positive,EURCHF to head higher towards 10130 and projec...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-to-head-...,FX Street,EUR/CHF has broken out above the sideways rang...,Positive,0.83
...,...,...,...,...,...,...,...,...,...,...
2286,2023-05-04 13:55:00,GBPUSD,Positive,GBPUSD holds steady near its highest level sin...,Haresh Menghani,https://www.fxstreet.com/news/gbp-usd-holds-st...,FX Street,The GBP/USD pair enters a bullish consolidatio...,Positive,0.63
2287,2023-05-04 14:12:00,EURCHF,Negative,EURCHF to break below the 097 level – Credit S...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-to-break...,FX Street,EUR/CHF has turned back lower over the past co...,Positive,0.47
2288,2023-05-04 14:17:00,EURUSD,Neutral,EURUSD Shortterm dips likely supported near 10...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-usd-short-te...,FX Street,The European Central Bank (ECB)downshifted to ...,Negative,-0.49
2289,2023-05-04 15:00:00,GBPUSD,Positive,GBPUSD Looking for an eventual final leg highe...,FXStreet Insights Team,https://www.fxstreet.com/news/gbp-usd-looking-...,FX Street,Economists at Credit Suisse discuss GBP outloo...,Neutral,0.43


In [30]:
sentiment_annotated_data["true_sentiment"].value_counts()

Neutral     815
Positive    767
Negative    709
Name: true_sentiment, dtype: int64

In [31]:
sentiment_annotated_data = sentiment_annotated_data.rename(columns={'text': 'sentence', 'true_sentiment': 'polarity'})

In [32]:
sentiment_annotated_data["polarity"]=sentiment_annotated_data.polarity.map({
    "Neutral": 0,
    "Positive": 1,
    "Negative": -1
})

In [33]:
sentiment_annotated_data["polarity"].value_counts()

 0    815
 1    767
-1    709
Name: polarity, dtype: int64

In [34]:
sentiment_annotated_data = sentiment_annotated_data.sample(frac=1, random_state=42)

In [35]:
# Splitting the data into training, validation, and test sets
train_annotated, temp = train_test_split(sentiment_annotated_data, test_size=0.3, random_state=42, stratify=sentiment_annotated_data["polarity"])
val_annotated, test_annotated = train_test_split(temp, test_size=0.5, random_state=42, stratify=temp["polarity"])

In [36]:
# # Split the data into a training set (80%) and a temporary set (20%)
# X_train_annotated, X_temp, y_train_annotated, y_temp = train_test_split(sentiment_annotated_data["sentence"], sentiment_annotated_data["polarity"], test_size=0.3, random_state=42,stratify= sentiment_annotated_data["polarity"])

# # Now, further split the temporary set into a training set (80%) and a validation set (20%)
# X_val_annotated, X_test_annotated, y_val_annotated, y_test_annotated = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42,stratify= y_temp)

In [37]:
train_annotated.shape

(1603, 10)

In [38]:
val_annotated.shape

(344, 10)

# Read  ALL_datafile

In [39]:
All_data=pd.read_csv("https://raw.githubusercontent.com/AnasElbattra/News-Understanding-NLP/main/Sentiment%20Analysis/Data/all-data.csv", encoding='latin-1', header=None)
All_data

,0,1
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [40]:
All_data = All_data.rename(columns={1: 'sentence', 0: 'polarity'})

In [41]:
All_data.polarity.value_counts()

neutral     2879
positive    1363
negative     604
Name: polarity, dtype: int64

In [42]:
All_data["sentence"]=All_data["sentence"].apply(Remove_line)


In [43]:
All_data["polarity"]=All_data.polarity.map({
    "neutral": 0,
    "positive": 1,
    "negative": -1
})

In [44]:
All_data.polarity.value_counts()

 0    2879
 1    1363
-1     604
Name: polarity, dtype: int64

In [45]:
All_data = All_data.sample(frac=1, random_state=42)

In [46]:
# Splitting the data into training, validation, and test sets
train_All_data, temp = train_test_split(All_data, test_size=0.3, random_state=42, stratify=All_data["polarity"])
val_All_data, test_All_data = train_test_split(temp, test_size=0.5, random_state=42, stratify=temp["polarity"])

In [47]:
train_All_data.shape

(3392, 2)

# Merge the three Dataframe

In [48]:
df_tain_selected=df_tain[["sentence","polarity"]]
train_gold_selected=train_gold[["sentence","polarity"]]
train_annotated_selected=train_annotated[["sentence","polarity"]]
train_All_data_selected=train_All_data[["sentence","polarity"]]


In [49]:
# Combine the DataFrames into one using pd.concat
merged_train = pd.concat([df_tain_selected,  train_All_data_selected], ignore_index=True)
merged_train

,sentence,polarity
0,Winner wrote that she had a 30-minute private ...,0
1,She also recently referred to President Trump ...,-1
2,She also recently referred to President Trump ...,-1
3,Hillary Clinton blamed the Democratic National...,-1
4,Hillary Clinton blamed the Democratic National...,-1
...,...,...
12126,"Also , CBA is to issue a benchmark , 10 year f...",0
12127,+_lemiste City is the environment for a knowle...,0
12128,GE is building the facility with wind power de...,0
12129,No financial or pricing details were disclosed .,0


In [50]:
df_validation_selected=df_validation[["sentence","polarity"]]
val_gold_selected=val_gold[["sentence","polarity"]]
val_annotated_selected=val_annotated[["sentence","polarity"]]
val_All_data_selected=val_All_data[["sentence","polarity"]]


In [51]:
# Combine the DataFrames into one using pd.concat
merged_val = pd.concat([df_validation_selected,val_All_data_selected], ignore_index=True)
merged_val

,sentence,polarity
0,In the 2015 Conservative Political Action Conf...,0
1,“He was willing to meet with industry to try t...,1
2,"""As far as the truck rental goes, those interv...",0
3,Then Trump didn’t actually let Priebus be a re...,-1
4,Then Trump didn’t actually let Priebus be a re...,-1
...,...,...
1065,"Water Treatment Products In Australia Today , ...",0
1066,Operating profit rose from EUR 1.94 mn to EUR ...,1
1067,"Even so , Tulikivi emissions are well below so...",0
1068,"Yesterday , Legrand issued its E300 million fi...",0


In [52]:
df_test_selected=df_test[["sentence","polarity"]]
# test_gold_selected=test_gold[["sentence","polarity"]]
test_annotated_selected=test_annotated[["sentence","polarity"]]
test_All_data_selected=test_All_data[["sentence","polarity"]]


In [104]:

merged_test = pd.concat([df_test_selected,  test_All_data_selected], ignore_index=True)
merged_test

,sentence,polarity
0,Though we do not know what other items Seth ma...,0
1,He won the Academy Award for Best Supporting A...,1
2,"""Yup, the whole birther movement was racist,"" ...",1
3,Ross's appointment will require a Senate confi...,0
4,Trustees Simone Boutet and Andrews voted no ea...,0
...,...,...
1525,DMASIA-16 August 2006-Benefon extends manufact...,1
1526,The first group of customers to be trained wil...,0
1527,The bridge will be built to the south of the e...,0
1528,These module products will be available for tr...,0


In [105]:
shuffled_test_df = merged_test.sample(frac=1, random_state=42)
shuffled_val_df = merged_val.sample(frac=1, random_state=42)
shuffled_train_df = merged_train.sample(frac=1, random_state=42)



In [106]:
clean_1_test = shuffled_test_df.groupby('polarity', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
clean_1_test = clean_1_test.sample(frac=1, random_state=42)
clean_1_test.dropna(subset=['sentence'], inplace=True)
clean_1_test

,sentence,polarity
1269,"The Costanza light , with an aluminum base and...",0
639,South Africa's parliament has merely postponed...,-1
1121,`` Stonesoft sees great promise in the future ...,1
451,These Democrats are all in the process of tryi...,0
688,He gave his advisers a tight deadline of the U...,0
...,...,...
1068,"Additionally , retailers can use the same hard...",0
118,"Despite the Cruz commotion, top party leaders ...",1
242,"The discussion was “not very good,” he said wi...",-1
1188,"The 10,000-odd square metre plot that Stockman...",0


In [56]:
shuffled_val_df.polarity.value_counts()

 0    568
 1    278
-1    224
Name: polarity, dtype: int64

In [57]:
x_train=shuffled_train_df["sentence"]
y_train=shuffled_train_df["polarity"]

In [58]:
x_valid=shuffled_val_df["sentence"]
y_valid=shuffled_val_df["polarity"]

In [107]:
x_test=clean_1_test["sentence"]
y_test=clean_1_test["polarity"]

In [60]:

# model_id ="lxyuan/distilbert-base-multilingual-cased-sentiments-student"
# model_id ="distilbert-base-uncased"
# model_id ="distilbert-base-uncased-finetuned-sst-2-english"

# model_id ="distilbert-base-uncased-distilled-squad"

# model_id ="Davlan/distilbert-base-multilingual-cased-ner-hrl"
# model_id ="xlnet-base-cased"
# model_id ="t5-small"
model_id = "cardiffnlp/twitter-roberta-base-sentiment"

# load the model from huggingface
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=3
                                                          #  , ignore_mismatched_sizes=True
                                                           )
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [61]:
train_encoding = tokenizer(x_train.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
valid_encoding = tokenizer(x_valid.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
test_encoding = tokenizer(x_test.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")


In [62]:
y_train = y_train.astype(int)
y_valid = y_valid.astype(int)
y_test = y_test.astype(int)


In [63]:
y_train

10143    1
3689     1
8950     0
7189    -1
7599     0
        ..
11964    0
5191    -1
5390     0
860     -1
7270     0
Name: polarity, Length: 12131, dtype: int64

In [64]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder on the target labels and transform y_train
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)
y_test_encoded = label_encoder.transform(y_test)



In [65]:
y_train.unique()

array([ 1,  0, -1])

In [66]:
{"1":0,"0":-1,"2":1}

{'1': 0, '0': -1, '2': 1}

In [67]:
y_train_encoded

array([2, 2, 1, ..., 1, 0, 1])

In [68]:
train_dataset=IMDbDataset(train_encoding,y_train_encoded)
valid_dataset=IMDbDataset(valid_encoding,y_valid_encoded)
test_dataset=IMDbDataset(test_encoding,y_test_encoded)


In [69]:
train_dataset

In [70]:
training_args = TrainingArguments(
    output_dir='./results',  # Fix the space before the directory path
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=10,
)


In [71]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
    # compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-3-a7e13701bd24>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,0.808400
20,0.738400
30,0.795300
40,0.719100
50,0.735900
60,0.712500
70,0.611400
80,0.659200
90,0.658300
100,0.764000


<ipython-input-3-a7e13701bd24>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=759, training_loss=0.6472109271917733, metrics={'train_runtime': 1191.6528, 'train_samples_per_second': 10.18, 'train_steps_per_second': 0.637, 'total_flos': 3191828870458368.0, 'train_loss': 0.6472109271917733, 'epoch': 1.0})

In [84]:
# output_dir="saved"
# tokenizer.save_pretrained(output_dir)

# model.save_pretrained(output_dir)

In [85]:
trainer.save_model()


trainer.save_state()


checkpoint_name = "test-trainer-deepspeed"
final_model_path = f"./llm04_fine_tuning/{checkpoint_name}"



trainer.save_model(output_dir=final_model_path)

In [86]:
# fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(output_dir)
# tokenizer_new = AutoTokenizer.from_pretrained(output_dir)


In [90]:

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(final_model_path)
# tokenizer_new = AutoTokenizer.from_pretrained(final_model_path)

In [108]:
inputs = tokenizer(x_test.tolist(),return_tensors="pt",padding='max_length', truncation=True, max_length=512)

In [109]:
import torch.nn.functional as F
with torch.no_grad():
    outputs =fine_tuned_model(**inputs)
    # print (outputs)
    prediction=F.softmax(outputs.logits, dim=1 )
    # print (prediction)

    labels= torch.argmax(prediction, dim=1 )
    # print(labels)
    classification= [fine_tuned_model.config.id2label[label_id] for label_id in labels.tolist()]

In [110]:
label_mapping = {"LABEL_1": 0, "LABEL_0": -1, "LABEL_2": 1}

classification_nnn= [label_mapping[label_id] for label_id in classification]

In [111]:
# classification_nnn

In [112]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(y_test, classification_nnn))

              precision    recall  f1-score   support

          -1       0.83      0.78      0.80        50
           0       0.70      0.78      0.74        50
           1       0.85      0.80      0.82        50

    accuracy                           0.79       150
   macro avg       0.79      0.79      0.79       150
weighted avg       0.79      0.79      0.79       150



In [113]:
# !pip install accelerate -U

In [114]:
text="A toddler with a bandaged head, limp and dusty, is carried into a Gaza hospital for treatment after an Israeli airstrike. Mourners bid farewell to a woman who was killed in southern Israel in the Oct.7 Hamas attack on a kibbutz.Scenes of destruction and pain endure on both sides as the latest conflict between Israel and Hamas stretches into a fourth week."

In [115]:
text=[text]

In [116]:
text

['A toddler with a bandaged head, limp and dusty, is carried into a Gaza hospital for treatment after an Israeli airstrike. Mourners bid farewell to a woman who was killed in southern Israel in the Oct.7 Hamas attack on a kibbutz.Scenes of destruction and pain endure on both sides as the latest conflict between Israel and Hamas stretches into a fourth week.']

In [117]:
One_csase = tokenizer(text,return_tensors="pt",padding='max_length', truncation=True, max_length=512)

In [118]:
import torch.nn.functional as F
with torch.no_grad():
    outputs =fine_tuned_model(**One_csase)

    prediction=F.softmax(outputs.logits, dim=1 )


    labels= torch.argmax(prediction, dim=1 )
    print(labels)
    classification= [fine_tuned_model.config.id2label[label_id] for label_id in labels.tolist()]

tensor([1])


In [119]:
label_mapping = {"LABEL_1": 0, "LABEL_0": -1, "LABEL_2": 1}

classification_nnn= [label_mapping[label_id] for label_id in classification]

In [120]:
classification_nnn

[0]

In [121]:
if (classification_nnn[0]==0):
  print ("Neutral")
if (classification_nnn[0]==-1):
  print ("Negative")
if (classification_nnn[0]==1):
  print ("Positive")

Neutral
